In [1]:
import pandas as pd
import numpy as np
import lucrum.datareader as ldr
from sklearn.utils import resample
from imblearn.over_sampling import SMOTE
from sklearn.metrics import precision_score
from lucrum.algo.crssovrml import MaCrssoverMl
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

from sklearn.ensemble import (RandomForestClassifier, AdaBoostClassifier, 
                              GradientBoostingClassifier, ExtraTreesClassifier)
from sklearn.svm import SVC

/home/delinvas/anaconda3/envs/ari5123/lib/python3.7/site-packages/sklearn/ensemble/weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


In [2]:
crsml = MaCrssoverMl(lead_ma="ema",
                     lead_time=5,
                     lag_ma="sma",
                     lag_time=8,
                     price="close")

# crsml.get_data(source="binance",
#                symbols="XRPUSDT",
#                start="5 May, 2018", 
#                end="18 Apr, 2019", 
#                interval="15m",
#                timezone="Europe/Malta")

# crsml.data.to_csv("dataxrp", sep='\t')

load_data = pd.read_csv('dataxrp.csv', sep='\t', dtype={"open": float})
load_data.drop("Unnamed: 0", axis=1, inplace=True)
crsml.set_data(load_data)

hist_price = ldr.get_data(source="binance",
             symbols="XRPUSDT",
             start="5 Apr, 2019", 
             end="18 Apr, 2019",
             interval="15m",
             timezone="Europe/Malta")

# display(crsml.data.head())
# crsml._gen_features(5)
# print(l)
# display(crsml.instances.head())
# display(crsml.instances.tail(100))

In [3]:
# Class to extend the Sklearn classifier
class SklearnHelper(object):
    def __init__(self, clf, seed=0, params=None):
        params['random_state'] = seed
        self.clf = clf(**params)

    def train(self, x_train, y_train):
        self.clf.fit(x_train, y_train)

    def predict(self, x):
        return self.clf.predict(x)
    
    def fit(self,x,y):
        return self.clf.fit(x,y)
    
    def feature_importances(self,x,y):
        print(self.clf.fit(x,y).feature_importances_)

In [4]:
lag=5
features = crsml._gen_features(lag, 1)
data = crsml.instances

display(data.head())

,open_time,open,high,low,close,close_time,trades,volume,ema_5,sma_8,ma_0,ma_1,ma_2,ma_3,ma_4,ma_5,crossover
12,2018-05-05 05:00:00+02:00,0.90094,0.90099,0.89831,0.89832,2018-05-05 05:14:59.999000+02:00,166,68384.99,0.899818,0.899966,0.000148,-0.000117,-0.000161,-0.000205,-0.000111,-0.000132,0
13,2018-05-05 05:15:00+02:00,0.89832,0.90234,0.89832,0.90234,2018-05-05 05:29:59.999000+02:00,212,85895.89,0.900659,0.901184,0.000525,0.000016,-0.000117,-0.000161,-0.000205,-0.000111,0
14,2018-05-05 05:30:00+02:00,0.90299,0.90299,0.89800,0.90193,2018-05-05 05:44:59.999000+02:00,328,198247.31,0.901083,0.901427,0.000345,0.000058,0.000016,-0.000117,-0.000161,-0.000205,1
15,2018-05-05 05:45:00+02:00,0.90193,0.90193,0.89833,0.90160,2018-05-05 05:59:59.999000+02:00,187,84758.84,0.901255,0.900946,-0.000309,0.000038,0.000058,0.000016,-0.000117,-0.000160,0
16,2018-05-05 06:00:00+02:00,0.90160,0.90550,0.90160,0.90550,2018-05-05 06:14:59.999000+02:00,234,144469.94,0.902670,0.901469,-0.001201,-0.000034,0.000038,0.000058,0.000016,-0.000117,0


In [6]:
# # best_i = 0
# # best_y = 0 

# # for i in range(1, 20):
# #     features = crsml._gen_features(i)
# #     data = crsml.instances

# #     # downsampling
# #     # Separate majority and minority classes
# #     data_majority = data[data["crossover"]==0]
# #     data_minority = data[data["crossover"]==1]

# #     # Downsample majority class
# #     df_majority_downsampled = resample(data_majority, 
# #                                        replace=False,    # sample without replacement
# #                                        n_samples=data.crossover.value_counts()[1], 
# #                                        random_state=314159)     # reproducible results

# #     # Combine minority class with downsampled majority class
# #     data_downsampled = pd.concat([df_majority_downsampled, data_minority])

# #     # Display new class counts
# #     #print(data_downsampled.crossover.value_counts())
# #     # display(data_downsampled)

# #     X = data_downsampled[features]
# #     y = data_downsampled["crossover"]

# #     X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.30)

# #     clf = RandomForestClassifier(n_estimators=100)
# #     clf.fit(X_train, y_train)
# #     y_pred = clf.predict(X_test)
    
# #     score = precision_score(y_test, y_pred, labels=[1,0], average='binary')
# #     if best_y < score:
# #         best_i = i 
# #         best_y = score
# #     #print(precision_score(y_test, y_pred, labels=[1,0], average='binary'))

# # print(best_i)
# # print(best_y)
lag = 8
features = crsml._gen_features(lag, 1)
data = crsml.instances
display(data.tail(100))

# downsampling
# Separate majority and minority classes
data_majority = data[data["crossover"]==0]
data_minority = data[data["crossover"]==1]

# Downsample majority class
df_majority_downsampled = resample(data_majority, 
                                   replace=False,    # sample without replacement
                                   n_samples=data.crossover.value_counts()[1], 
                                   random_state=314159)     # reproducible results

# Combine minority class with downsampled majority class
data_downsampled = pd.concat([df_majority_downsampled, data_minority])

# Display new class count
#print(data_downsampled.crossover.value_counts())
# display(data_downsampled)

X = data_downsampled[features]
print(features)
# X = data_downsampled[["sma_8", "ema_5"]]
y = data_downsampled["crossover"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.30)
# clf = ExtraTreesClassifier(n_jobs=-1, n_estimators=500,max_depth=8,min_samples_leaf=2)
clf = RandomForestClassifier(n_estimators=100)
# clf = RandomForestClassifier(n_estimators=500, 
#                              warm_start=True, 
#                              max_depth=6, 
#                              min_samples_leaf=2, 
#                              max_features="sqrt")
# clf = AdaBoostClassifier(n_estimators= 500, learning_rate = 0.75)
# clf = GradientBoostingClassifier(n_estimators=500,max_depth=5,min_samples_leaf=2)
# clf = SVC(kernel= 'linear', C=10.45)

clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
score = precision_score(y_test, y_pred, labels=[1,0], average='binary')

probs_y = clf.predict_proba(X_test) 
probs_y_pred = np.array([1 if x[1] >= 0.6 else 0 for x in probs_y])
score_2 = precision_score(y_test, probs_y_pred, labels=[1,0], average='binary')

print("Pred: {0} Pred Thres: {1}".format(score, score_2))
# print(score)
print(probs_y_pred[probs_y_pred == 1].shape)

# #     if best_y < score:
# #         best_i = i 
# #         best_y = score
# #     #print(precision_score(y_test, y_pred, labels=[1,0], average='binary'))

# # print(best_i)
# # print(best_y)

,open_time,open,high,low,close,close_time,trades,volume,ema_5,sma_8,ma_0,ma_1,ma_2,ma_3,ma_4,ma_5,ma_6,ma_7,ma_8,crossover
33165,2019-04-17 01:00:00+02:00,0.32214,0.32279,0.32174,0.32266,2019-04-17 01:14:59.999000+02:00,348,398836.3,0.321875,0.321544,-0.000331,-2.523817e-05,9.790479e-06,2.842091e-05,2.986858e-05,-1.143168e-06,-2.031195e-05,1.426260e-05,3.634459e-05,0
33166,2019-04-17 01:15:00+02:00,0.32263,0.32300,0.32236,0.32250,2019-04-17 01:29:59.999000+02:00,201,185006.0,0.322083,0.321669,-0.000414,-6.438541e-05,-2.523553e-05,9.789455e-06,2.841793e-05,2.986545e-05,-1.143048e-06,-2.030983e-05,1.426110e-05,0
33167,2019-04-17 01:30:00+02:00,0.32249,0.32479,0.32249,0.32449,2019-04-17 01:44:59.999000+02:00,640,950636.0,0.322885,0.321994,-0.000892,-8.058644e-05,-6.437436e-05,-2.523120e-05,9.787775e-06,2.841306e-05,2.986032e-05,-1.142852e-06,-2.030634e-05,0
33168,2019-04-17 01:45:00+02:00,0.32448,0.32500,0.32379,0.32379,2019-04-17 01:59:59.999000+02:00,388,484795.4,0.323187,0.322324,-0.000863,-1.733151e-04,-8.055731e-05,-6.435109e-05,-2.522208e-05,9.784237e-06,2.840279e-05,2.984953e-05,-1.142439e-06,0
33169,2019-04-17 02:00:00+02:00,0.32400,0.32540,0.32314,0.32382,2019-04-17 02:14:59.999000+02:00,699,921183.3,0.323398,0.322694,-0.000704,-1.676898e-04,-1.732253e-04,-8.051557e-05,-6.431775e-05,-2.520901e-05,9.779167e-06,2.838807e-05,2.983406e-05,0
33170,2019-04-17 02:15:00+02:00,0.32378,0.32448,0.32322,0.32380,2019-04-17 02:29:59.999000+02:00,423,430358.5,0.323532,0.323047,-0.000484,-1.367116e-04,-1.675770e-04,-1.731088e-04,-8.046141e-05,-6.427448e-05,-2.519205e-05,9.772588e-06,2.836897e-05,0
33171,2019-04-17 02:30:00+02:00,0.32358,0.32402,0.32291,0.32332,2019-04-17 02:44:59.999000+02:00,334,360972.6,0.323461,0.323311,-0.000150,-9.397746e-05,-1.366019e-04,-1.674425e-04,-1.729699e-04,-8.039682e-05,-6.422289e-05,-2.517183e-05,9.764744e-06,1
33172,2019-04-17 02:45:00+02:00,0.32332,0.32332,0.32187,0.32201,2019-04-17 02:59:59.999000+02:00,418,432049.8,0.322978,0.323299,0.000321,-2.908637e-05,-9.389759e-05,-1.364858e-04,-1.673002e-04,-1.728228e-04,-8.032849e-05,-6.416831e-05,-2.515044e-05,0
33173,2019-04-17 03:00:00+02:00,0.32202,0.32258,0.32062,0.32111,2019-04-17 03:14:59.999000+02:00,543,928340.7,0.322355,0.323105,0.000750,6.220774e-05,-2.906698e-05,-9.383500e-05,-1.363948e-04,-1.671887e-04,-1.727076e-04,-8.027495e-05,-6.412554e-05,0
33174,2019-04-17 03:15:00+02:00,0.32093,0.32200,0.32050,0.32174,2019-04-17 03:29:59.999000+02:00,412,533990.4,0.322150,0.323010,0.000860,1.451921e-04,6.218315e-05,-2.905549e-05,-9.379792e-05,-1.363409e-04,-1.671226e-04,-1.726394e-04,-8.024322e-05,0


['ma_0', 'ma_1', 'ma_2', 'ma_3', 'ma_4', 'ma_5', 'ma_6', 'ma_7', 'ma_8']
Pred: 0.7755498059508409 Pred Thres: 0.8236641221374046
(1310,)
